# Исследование надёжности заёмщиков

**Цель исследования** - изучение влияния различных факторов (наличие и количество детей или их отсутствие, семейное положение, уровень совокупного дохода, цель кредитования) на возможность погашения кредита в срок, необходимое для использования полученных результатов при построении модели кредитного скоринга.

**Задачи исследования**:

1. Изучить предоставленные банком данные о платежеспособности клиентов.
2. Подготовить данные к проведению исследования.
3. Оценить влияние различных факторов на возможность погашения кредита в срок.
4. Дать рекомендации по улучшению построения модели кредитного сокринга.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
#импортируем библиотеку Pandas и открываем файл

import pandas as pd
data = pd.read_csv('/datasets/data.csv')

#просмотрим информацию о таблице
data.info()

#выведем на экран 10 первых значений для изучения данных

data.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Изучив информацию о таблице, видно, что во всех столбцах встречаются пропущенные значения (как категориальные, так и количественные). В столбце days_employed помимо пропущенных значений встречаются отрицательные значения, которые из-за отсутствия округления, трудно визуально воспринимать. Более того, в столбце days_employed возможно некорректная единица измерения. Но, поскольку этот столбец в дальнейшем не будет участвовать в расчетах, можно не придавать этому особого значения. В столбце total_income также трудно воспринимать значения из-за отсутствия округления. Пропуски могут быть вызваны разными факторами: первоначальное предоставление некорректных данных, человеческий фактор, технологические проблемы при выгрузке данных. Для проведения расчетов необходимо провести предобработку данных.


## Шаг 2. Предобработка данных

### Обработка пропусков

Для дальнейших расчетов нам необходимы следующие столбцы: children, family-status,family_status_id,income_type,total_income. В столбце days_employed заполним пропуски, удалять их не нужно, поскольку в строках других столбцов может быть важная для исследования информация. 

In [2]:
#для определения пропущенных значений в столбце days_employed используем методы loc и isna()

data.loc[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [3]:
#с помощью метода fillna() заполним пропущенные значения в столбце days_employed на нулевое значение, 
#т.к. этот столбец не участвует в расчете влияния на срок погашения кредита

data['days_employed'] = data['days_employed'].fillna(0)

#посчитаем количество пропущенных значений в столбце days_employed с помощью функции len()

len(data[data['days_employed'].isna()])



0

In [4]:
#для определения пропущенных значений в столбце total_income используем методы loc и isna()

data.loc[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,0.0,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,0.0,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,0.0,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,0.0,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Поскольку столбец total_income содержит количественные значения, которые участвуют в оценке платежеспособности клиентов банка, мы не можем заполнить пропущенные строки нулевым значением или пустой строкой. Пропущенные значения общего дохода заполним медианным значением по категориям плательщиков, поскольку среднее значение можеть сильно исказить результаты исследования.

In [5]:
#сгруппируем данные total_income по категориям income_type и заменим пропуски на медианные значения 
#с помощью метода transform(). Результат сохраним в переменной data['total_income']

data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

#посчитаем количество пропущенных значений в столбце income_total

len(data[data['total_income'].isna()])


0

Столбец children содержит данные о количестве детей и содержит пропущенные значения. Вероятно, не все клиенты банка хотели указывать данные о наличии детей. Это могло быть причиной возникновения пропущенных значений. Для дальнейших расчетов заменим пропуски средним значением количества детей по категориям клиентов.

In [6]:
#находим среднее количество детей по категориям income_type

children_med_by_type = data.groupby('income_type')['children'].mean()

#заменим пропущенные значения в столбце children средним значением

data['children'].fillna(children_med_by_type, inplace = True)

#проверим количество пропущенных значений в столбце children

len(data[data['children'].isna()])


0

Пропущенные значения в столбцах family_status и family_status_id являются категориальными. Мы не можем заполнить их каким-либо значением по умолчанию, поскольку точно не знаем какое значение выбрать. Возможно, причиной пропусков стало то, что информация о семейном статусе личная и для клиента банка не очевидная, чтобы указывать её в анкете на получение кредита. В любом случае, какая-то часть выборки не будет учитываться из-за пропущенных значений в этих двух столбцах, поэтому заполним их нулевым значением.

In [7]:
#заполним пропущенные значения в столбце family_status нулевым значением

data['family_status'] = data['family_status'].fillna(0)

#посчитаем количество пропущенных строк

len(data[data['family_status'].isna()])

0

In [8]:
#заполним пропущенные значения в столбце family_status_id нулевым значением

data['family_status_id'] = data['family_status_id'].fillna(0)

#посчитаем количество пропущенных строк

len(data[data['family_status_id'].isna()])

0

In [9]:
#выведем на экран 10 первых строк, чтобы убедиться, что пропусков в столбцах больше нет

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [10]:
#выведем на экран 10 последних строк, чтобы убедиться, что пропусков в столбцах больше нет

data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


**Вывод**

В зависимости от типа значений (категориальные или количественные) пропущенные значения были заменены в столбцах, которые необходимы для проведения исследования. Для заполнения пропущенных количественных значений, влияющих на результаты исследования (столбцы total_income, children) был применены методы их замены на медианные значения (столбец со значением общего дохода) и средние значения (количество детей) в разрезе категорий клиентов банка. В столбцах с пропущенными категориальными значениями (family_status, family_status_id), они были заполнены нулевым значением, поскольку мы не знаем точно о том, какое значение из указанных столбцов для заполнения подошло бы лучше. Данные в таблице готовы для дальнейшей предобработки.

### Замена типа данных

Из данных о таблице видно, что столбец days_employed содержит данные типа string, хотя по факту это вещественные числа с отрицательным значением. Необходимо привести данные столбца к нужному типу данных и корректным значениям.

In [11]:
#преобразуем значения string во float с помощью метода to_numeric

data['days_employed'] = pd.to_numeric(data['days_employed']) 

#проверим изменился ли тип данных

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [12]:
#заменим отрицательные значения в столбце days_employed положительными с помощью функции abs()
# и изменим данные с float на int. Результат сохраним в переменной data['days_employed']

data['days_employed'] = data['days_employed'].abs().astype('int')

#выведем первые 10 строк на экран

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [13]:
#изменим данные с float на int. Результат сохраним в переменной data['total_income']

data['total_income'] = data['total_income'].astype('int')

#выведем первые 10 строк на экран

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Среди предоставленных банком данных о платежеспособности клиентов в разрезе значений трудового стажа были замечены следующие ошибки в данных:
1. Несоответствие типа данных со значениями;
2. Отрицательные значения.

Также среди значений о трудовом стаже и совокупном доходе были замечены вещественные числа, которые визуально усложняют восприятие данных. С помощью методов, доступных в библиотеке pandas, значения в столбцах с некорректными данными были приведены в надлежащий вид, необходимый для проведения дальнейшей предобработки.


### Обработка дубликатов

In [14]:
#посчитаем количество дубликатов в таблице

data.duplicated().sum()

54

Из результата применения методов duplicated() и sum() видно, что датафрейм содержит дубликаты. Причинами их появления могут быть ошибки ввода данных, пропуск данных, наличие сокращений или аббревиатур, использование сочетания верхниго и нижнего регистра. Чтобы убедиться, что дубликаты удалены, необходимо просмотреть уникальные значения в столбцах, которые нам нужны для проведения исследования: children,family_status,family_status_id.

In [15]:
#изучим уникальные значения и частоту их встречаемости в столбце children

data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Из результата применения метода value_counts() видно,что в столбце children встречаются аномальные значения: -1 и 20. Можно предположить, что это должны были быть значения 1 и 20, но точно мы этого не знаем, поэтому удалим эти строки. В целом, удаление строк с аномальными значениями не отразится критично на результатах исследования, т.к. их удельный вес в выборке мал (не достигает 1%). Для удаления строк будем использовать фильтр, т.к. нам известны только значения, которые нужно удалить, но нет данных о строках, в которых они находятся.

In [16]:
#отфильтруем (удалим) строки со значением 20 в столбце children

data = data.loc[data['children']!=20]

#отфильтруем (удалим) строки со значением -1 в столбце children

data = data.loc[data['children']!=-1]

#просмотрим уникальные значения столбца children

data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [17]:
#просмотрим уникальные значения в столбце family_status

data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [18]:
#просмотрим уникальные значения в столбце family_status_id

data['family_status_id'].value_counts()

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

In [19]:
#удалим дубликаты в таблице

data = data.drop_duplicates().reset_index(drop = True)

In [20]:
#убедимся, что в таблице нет дубликатов

data.duplicated().sum()

0

**Вывод**

При проведении проверки данных, были обнаружены дубликаты, которые были удалены. Также при проверке данных в столбце с информацией о количестве детей были обнаружены некорректные данные: отрицательное (-1) и аномальное (20) значения. Поскольку строки с некорректными значениями занимают порядка 1% из выборки, они были также удалены. 

### Лемматизация

По условиям задачи необходимо провести лемматизацию значений в стролбце purpose. Для привидения слова к словарной форме используется библиотека pymystem3 и метод lemmatize(). Результат лемматизации необходим для понимания целей взятия клиентами банка кредита, для того, чтобы по целям можно было сформировать определенные категории.

In [21]:
#импортирцем библиотеку pymystem3 

from pymystem3 import Mystem
m = Mystem()

In [22]:
#просматриваем список уникальных значений в столбце purpose. Результат сохраняем в переменной purpose

purpose = data['purpose'].unique()

#выводим список уникальных значений на экран

purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [23]:
#преобразуем значения столбца purpose в строку для проведения лемматизации. Результат сохраняем в переменной purpose_list

purpose_list = """"покупка жилья, приобретение автомобиля,
       дополнительное образование, сыграть свадьбу,
       операции с жильем, образование, на проведение свадьбы,
       покупка жилья для семьи, покупка недвижимости,
       покупка коммерческой недвижимости, покупка жилой недвижимости,
       строительство собственной недвижимости, недвижимость,
       строительство недвижимости, на покупку подержанного автомобиля,
       на покупку своего автомобиля,
       операции с коммерческой недвижимостью,
       строительство жилой недвижимости, жилье,
       операции со своей недвижимостью, автомобили,
       заняться образованием, сделка с подержанным автомобилем,
       получение образования, автомобиль, свадьба,
       получение дополнительного образования, покупка своего жилья,
       операции с недвижимостью, получение высшего образования,
       свой автомобиль, сделка с автомобилем,
       профильное образование, высшее образование,
       покупка жилья для сдачи, на покупку автомобиля, ремонт жилью,
       заняться высшим образованием"""

#передаем переменной lemmas значения списка purpose_list для лемматизации

lemmas = m.lemmatize(purpose_list)

#выводим результат лемматизации на экран

lemmas

['"',
 'покупка',
 ' ',
 'жилье',
 ', ',
 'приобретение',
 ' ',
 'автомобиль',
 ',\n',
 '       ',
 'дополнительный',
 ' ',
 'образование',
 ', ',
 'сыграть',
 ' ',
 'свадьба',
 ',\n',
 '       ',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 ', ',
 'образование',
 ', ',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 ',\n',
 '       ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 ', ',
 'покупка',
 ' ',
 'недвижимость',
 ',\n',
 '       ',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ', ',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ',\n',
 '       ',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 ', ',
 'недвижимость',
 ',\n',
 '       ',
 'строительство',
 ' ',
 'недвижимость',
 ', ',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 ',\n',
 '       ',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 ',\n',
 '       ',
 'операция',
 ' ',
 'с',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ',\n',
 '       

In [24]:
#для подсчета частоты встречаемости лемм импортируем Counter

from collections import Counter

#выводим результат подсчета на экран

Counter(lemmas)

Counter({'"': 1,
         'покупка': 10,
         ' ': 59,
         'жилье': 7,
         ', ': 19,
         'приобретение': 1,
         'автомобиль': 9,
         ',\n': 18,
         '       ': 18,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Исходя из определения лемм видно, что есть 4 основные цели получения кредита, связанных с покупкой недвижимости или жилья, покупкой автомобиля, получения образования, организации свадьбы. На основе этих данных напишем функцию для замены значений в столбце согласно выбранным категориям.

In [25]:
#объявляем функцию lemma_function

def lemma_function(purpose):
    lemma = m.lemmatize(purpose) #сохраняем результат лемматизации в переменную lemma
    if ('недвижимость' in lemma) or ('жилье' in lemma): #объявляем условие 
        lem_name = 'недвижимость' #если значение соответствует лемме, сохраняем результат в переменной lem_name
        return lem_name           # и возвращаем результат с помощью функции return
    
    elif 'автомобиль' in lemma:
        lem_name = 'автомобиль'
        return lem_name
    
    elif 'образование' in lemma:
        lem_name = 'образование'
        return lem_name
    
    elif 'свадьба' in lemma:
        lem_name = 'свадьба'
        return lem_name
    
    else:
        return 'другое'
    

In [26]:
#проверим работу функции lemma_function

lemma_function('образование')

'образование'

In [27]:
lemma_function('недвижимость')

'недвижимость'

In [28]:
#применяем функцию lemma_function к значениям в столбце purpose

purpose = data['purpose'].apply(lemma_function)

#заменяем значения в столбце purpose на лемматизированные

data['purpose'] = purpose

#выведем первые 10 строк таблицы на экран

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


**Вывод**

В результате изучения данных о целях кредитования, при проведении лемматизации выяснилось, что все цели можно поделить на 4 категории: операции с недвижимостью или жильем, получение образования, финансирование проведения свадьбы и приобретение автомобиля (или иные операции с автомобилем). 

### Категоризация данных

Поскольку нам нужно найти зависимость между погашением кредита в срок и такими факторами, как наличие детей, семейное положение, уровень дохода и цель кредитования, необходимо категоризировать следующие столбцы: family_status, family_status_id, children, total_income, purpose. Для группировки значений будем использовать метод groupby(), для подсчета значений в столбцах метод count() в сочетании с методом agg().

In [29]:
#сгруппируем данные о задолженности по столбцам family_status_id, family_status 
#и посчитаем количество должников по категориям. Результат сохраним в новой таблице data_family

data_family = data.groupby(['family_status_id','family_status','debt']).agg({'debt':'count'})


#выведем таблицу на экран

data_family

debt
family_status_id family_status         debt       
0                женат / замужем       0     11339
                                       1       927
1                гражданский брак      0      3761
                                       1       385
2                вдовец / вдова        0       888
                                       1        63
3                в разводе             0      1105
                                       1        84
4                Не женат / не замужем 0      2523
                                       1       273

In [30]:
#посчитаем процент вероятности возврата кредита в срок в зависимости от семейного положения 

data_family = (data_family['debt'] / (data_family.groupby('family_status_id')['debt'].transform('sum')))*100 

#выведем таблицу на экран

data_family

family_status_id  family_status          debt
0                 женат / замужем        0       92.442524
                                         1        7.557476
1                 гражданский брак       0       90.713941
                                         1        9.286059
2                 вдовец / вдова         0       93.375394
                                         1        6.624606
3                 в разводе              0       92.935240
                                         1        7.064760
4                 Не женат / не замужем  0       90.236052
                                         1        9.763948
Name: debt, dtype: float64

**Вывод**

Исходя из данных о наличии задолженности в разрезе семейного положения клиентов банка, можно сказать, что вероятность погашения кредита в срок среди тех, кто женат или замужем, вдовцов/вдов и тех, кто в разводе, выше, чем среди тех, кто холост или не замужем и тех, кто живет в гражданском браке.

In [31]:
#сгруппируем данные о задолженности по столбцу children и посчитаем количество должников по категориям. 
#Результат сохраним в новой таблице data_children

data_children = data.groupby(['children','debt']).agg({'debt':'count'})

#выведем на экран новую таблицу data_children

data_children

debt
children debt       
0        0     13044
         1      1063
1        0      4365
         1       444
2        0      1858
         1       194
3        0       303
         1        27
4        0        37
         1         4
5        0         9

In [32]:
#посчитаем процент вероятности возврата кредита в срок в зависимости от наличия, 
#количества или отсутствия детей

data_children = (data_children['debt'] / (data_children.groupby('children')['debt'].transform('sum')))*100 

#выведем таблицу на экран

data_children

children  debt
0         0        92.464734
          1         7.535266
1         0        90.767311
          1         9.232689
2         0        90.545809
          1         9.454191
3         0        91.818182
          1         8.181818
4         0        90.243902
          1         9.756098
5         0       100.000000
Name: debt, dtype: float64

**Вывод**

Исходя из группировки данных о задолженности в зависимости от наличия детей видно, что и при наличии 1-4 детей и при их отсутствии, доля тех, кто погашает кредит в срок 90,2 - 92,5%. При наличии пятерых детей вероятность погашения кредита в срок максимальная, однако в выборке отсутствует информация о непогашенных в срок кредитах у клиентов с наличием пяти детей. В целом, можно сказать, что наличие или отсутствие детей не влияет на возможность погашения кредита в срок, поскольку наличие 3 и более детей встречается гораздо реже, чем 1-2 ребенка или их отсутствие.

Для оценки влияния уровня дохода на погашение кредита в срок, нам необходимо категоризировать данные о совокупном доходе. Для этого найдем максимальное и минимальное значения в столбце total_income.

In [33]:
#находим максимальное значение совокупного дохода

data['total_income'].max()

2265604

In [34]:
#находим минимальное значение совокупного дохода

data['total_income'].min()

20667

Разбежка между минимальным и максимальным значениями совокупного дохода внушительная. Возможно данные некорректны и необходимо уточнить у разработчиков соответствуют ли значения действительности. Предположим, что данные корректны. Для дальнейшей оценки влияния уровня дохода на срок погашения кредита разделим уровень доходов на несколько категорий: низкий, средний, высокий и очень высокий доход. Для этого напишем функцию income_id.

In [35]:
def income_id(total_income): #объявляем функцию income_id
    if 20000 <= total_income <= 99999: #объявляем условие
        return 'низкий' #возвращаем значение, если оно удовлетворяет условию
    
    elif 100000 <= total_income <= 200000:
        return 'средний'
    
    elif 200001 <= total_income <= 500000:
        return 'высокий'
    else:
        return 'очень высокий'

In [36]:
#объявим функцию income_id для проверки корректности работы

income_id(75000)

'низкий'

In [37]:
income_id(750000)

'очень высокий'

In [38]:
#применим функцию income_id к значениям в столбце total_income с помощью метода apply()

income_id = data['total_income'].apply(income_id)

#создадим новый столбец income_id с результатами работы функции income_id 

data['income_id'] = income_id

#выведем первые 10 строк таблицы на экран

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,средний
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,высокий
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,средний
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,средний


In [39]:
#сгруппируем данные о задолженности по столбцу income_id и посчитаем количество должников по категориям. 
#Результат сохраним в новой таблице data_income

data_income = data.groupby(['income_id','debt']).agg({'debt':'count'})

#выведем таблицу на экран

data_income

debt
income_id     debt       
высокий       0      4475
              1       342
низкий        0      4090
              1       354
очень высокий 0       208
              1        14
средний       0     10843
              1      1022

In [40]:
#посчитаем процент вероятности возврата кредита в срок в зависимости от уровня дохода

data_income = (data_income['debt'] / (data_income.groupby('income_id')['debt'].transform('sum')))*100 

#выведем таблицу на экран

data_income

income_id      debt
высокий        0       92.900145
               1        7.099855
низкий         0       92.034203
               1        7.965797
очень высокий  0       93.693694
               1        6.306306
средний        0       91.386431
               1        8.613569
Name: debt, dtype: float64

**Вывод**

Исходя из группировки данных по уровню дохода видно, что вероятность невозврата кредита в срок у клиентов со средним уровнем дохода - 8,6%. У остальных категорий этот показатель чуть ниже (7,1 - 8%). Поскольку данные о совокупном доходе требуют уточнения, в данном контексте исследование может выдавать некорректный результат.

In [41]:
#сгруппируем данные о задолженности по столбцу purpose и посчитаем количество должников по категориям. 
#Результат сохраним в новой таблице data_purpose

data_purpose = data.groupby(['purpose','debt']).agg({'debt':'count'})

#выведем таблицу data_purpose на экран

data_purpose

debt
purpose      debt      
автомобиль   0     3881
             1      400
недвижимость 0     9974
             1      780
образование  0     3620
             1      369
свадьба      0     2141
             1      183

In [42]:
#посчитаем процент вероятности возврата кредита в срок в зависимости от целей кредитования

data_purpose = (data_purpose['debt'] / (data_purpose.groupby('purpose')['debt'].transform('sum')))*100 

#выведем таблицу на экран

data_purpose

purpose       debt
автомобиль    0       90.656389
              1        9.343611
недвижимость  0       92.746885
              1        7.253115
образование   0       90.749561
              1        9.250439
свадьба       0       92.125645
              1        7.874355
Name: debt, dtype: float64

**Вывод**

Из результата группировки наличия задолженности в зависимости от целей кредитования видно, что вероятность возврата кредита в срок при его получении для операций с недвижимостью и проведения свадьбы составляет 92,1 - 92,7%. Вероятность возврата кредита в срок при его получении для покупки автомобиля или получения образования, ниже - 90,7%.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Нет, наличие или отсутствие детей не оказывает существенного влияния на вероятность возврата кредита в срок.

**Вывод**

Исходя из группировки данных о задолженности в зависимости от наличия детей видно, что и при наличии 1-4 детей и при их отсутствии, доля тех, кто погашает кредит в срок 90,2 - 92,5%. При наличии пятерых детей вероятность погашения кредита в срок максимальная, однако в выборке отсутствует информация о непогашенных в срок кредитах у клиентов с наличием пяти детей. В целом, можно сказать, что наличие или отсутствие детей не влияет на возможность погашения кредита в срок, поскольку наличие 3 и более детей встречается гораздо реже, чем 1-2 ребенка или их отсутствие.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Есть некоторая зависимость.

**Вывод**

Исходя из данных о наличии задолженности в разрезе семейного положения клиентов банка, можно сказать, что вероятность погашения кредита в срок среди тех, кто женат или замужем, вдовцов/вдов и тех, кто в разводе, выше, чем среди тех, кто холост или не замужем и тех, кто живет в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Если считать, что данные корректны, то есть.

**Вывод**

Исходя из группировки данных по уровню дохода видно, что вероятность невозврата кредита в срок у клиентов со средним уровнем дохода - 8,6%. У остальных категорий этот показатель чуть ниже (7,1 - 8%). Поскольку данные о совокупном доходе требуют уточнения, в данном контексте исследование может выдавать некорректный результат.

- Как разные цели кредита влияют на его возврат в срок?

Есть отличия 

**Вывод**

Из результата группировки наличия задолженности в зависимости от целей кредитования видно, что вероятность возврата кредита в срок при его получении для операций с недвижимостью и проведения свадьбы составляет 92,1 - 92,7%. Вероятность возврата кредита в срок при его получении для покупки автомобиля или получения образования, ниже - 90,7%.

## Шаг 4. Общий вывод

Исходя из предоставленных банком данных о платежеспособности клиентов и их изучения в разрезе влияния различных факторов на вероятность погашения кредита в срок, при построении модели кредитного скоринга необходимо учитывать следующее:

1. В зависимости от семейного положения клиента, вероятность погашения кредита в срок выше среди тех, кто женат или замужем, является вдовцом или вдовой и тех, кто находится в разводе, чем среди тех, кто холост или не замужем или состоит в гражданском браке: 92,4 - 93,4% против 90,2 - 90,7%.


2. В разрезе от наличия, количества или отсутствия детей, самая высокая вероятность возврата кредита в срок среди тех, кто не имеет детей - 92,5%. Среди клиентов, у которых есть 1-4 детей вероятность возврата кредита чуть ниже и составляет  - 90,7 - 91,8%. Максимальное значение вероятности возврата кредита среди клиентов, имеющих детей, было зафиксировано среди тех, у кого 5 детей, однако данные скорее всего некорректные, поскольку нет значений о невозврате кредита этой категорией клиентов. На клиентов, у которых 3 и более детей ориентироваться не стоит, поскольку эта категория встречается гораздо реже, чем те клиенты, у которых 1-2 ребенка или дети отсутствуют.


3. В зависимости от уровня совокупного дохода, самая низкая вероятность возврата кредита в срок у клиентов со средним уровнем дохода - 8,6%. По всем остальным категориям этот показатель колеблется на уровне 7,1 - 8,0%. Поскольку в данных о совокупном доходе сложно выделить категории из-за большой разбежки между минимальным и максимальным значением, категории были выделены вручную, что может исказить результат исследования. 


4. Цели кредитования тоже оказывают некоторое влияние на срок возврата кредита. Доля клиентов, вероятность возврата кредита в срок которых максимальная - 92, 1 - 92,7%, берут кредит для покупки недвижимости или проведения свадьбы. Среди тех, кто получает кредит с целью получения образования или покупки автомобиля, верятность возврата кредита в срок чуть ниже и составляет 90,7%.
